In [159]:
pip install bs4
pip install requests

SyntaxError: invalid syntax (<ipython-input-159-1dcc8df3f8f2>, line 1)

In [105]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import html

In [12]:
def scrape_wiki(wikiUrl):
        response = requests.get(wikiUrl)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        heading_tags = ["h1", "h2", "h3","h4"]
        header=[]
        for head in soup.findAll(heading_tags):
               header.append(head.text)
        for h in header:
                print(h.rstrip("\n").lstrip("\n"))
scrape_wiki('https://en.wikipedia.org/wiki/Main_Page')

Main Page
From today's featured article
Did you know ...
In the news
On this day
From today's featured list
Today's featured picture
Other areas of Wikipedia
Wikipedia's sister projects
Wikipedia languages
Navigation menu
Personal tools
Namespaces
Variants
Views
More
Search
Navigation
Contribute
Tools
Print/export
In other projects
Languages


In [45]:
def scrape_Imdb(imdbUrl):
        response = requests.get(imdbUrl)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        name=[]
        rating=[]
        year=[]
        for head in soup.findAll(class_='lister-item-content'):
            name.append(head.a.text)
        for y in soup.findAll(class_='lister-item-year text-muted unbold'):
            year.append(y.text)
        for rate in soup.findAll(class_="inline-block ratings-imdb-rating"):
            rating.append(rate.text.replace('\n','').strip())
        if(len(name)<100):
            startpage=len(name)+1
            imdbUrl2=imdbUrl.replace('start=0','start={}'.format(startpage))
            #print(imdbUrl2)
            response = requests.get(imdbUrl2)
            #print(response.status_code)
            html = response.text
            soup=BeautifulSoup(html)
            prettyhtml=soup.prettify()
            for head in soup.findAll(class_='lister-item-content'):
                name.append(head.a.text)
            for y in soup.findAll(class_='lister-item-year text-muted unbold'):
                year.append(y.text)
            for rate in soup.findAll(class_="inline-block ratings-imdb-rating"):
                rating.append(rate.text.replace('\n','').strip())
        MovieReviews=pd.DataFrame({
          "MovieName" :name,
            "Imdb Rating":rating,
            "ReleaseYear":year
        }) 
        return MovieReviews
        
df=scrape_Imdb('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc&start=0&ref_=adv_nxt')
df.head()

,MovieName,Imdb Rating,ReleaseYear
0,The Shawshank Redemption,9.3,(1994)
1,The Godfather,9.2,(1972)
2,The Dark Knight,9.0,(2008)
3,The Godfather: Part II,9.0,(1974)
4,12 Angry Men,9.0,(1957)


In [67]:
def scrape_ImdbIndianMovies(imdbUrlI):
        response = requests.get(imdbUrlI)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        name=[]
        rating=[]
        year=[]
        for head in soup.findAll('h3',class_='lister-item-header'):
            name.append(head.a.text)
        for y in soup.findAll(class_='lister-item-year text-muted unbold'):
            year.append(y.text)
        for rate in soup.findAll('div',class_="ipl-rating-star small"):
            rating.append(rate.text.replace('\n','').strip())
        IndianMovieReviews=pd.DataFrame({
          "MovieName" :name,
           "Imdb Rating":rating,
            "ReleaseYear":year
        }) 
        return IndianMovieReviews

df=scrape_ImdbIndianMovies('https://www.imdb.com/list/ls056092300/')
print(df)

                            MovieName Imdb Rating ReleaseYear
0                     Ship of Theseus         8.1      (2012)
1                              Iruvar         8.5      (1997)
2                     Kaagaz Ke Phool           8      (1959)
3   Lagaan: Once Upon a Time in India         8.1      (2001)
4                     Pather Panchali         8.6      (1955)
..                                ...         ...         ...
95                        Apur Sansar         8.5      (1959)
96                        Kanchivaram         8.1      (2008)
97                    Monsoon Wedding         7.4      (2001)
98                              Black         8.2      (2005)
99                            Deewaar         8.1      (1975)

[100 rows x 3 columns]


In [123]:
def bookReview(bookreviewurl):
    response = requests.get(bookreviewurl)
    #print(response.status_code)
    reviewhtml = response.text
    soup=BeautifulSoup(reviewhtml)
    prettyhtml=soup.prettify()
    bookName=[]
    authorName=[]
    genre=[]
    reviewLink=[]
    reviews=[]
    BookReviews = {"BookName":[],"Author":[],"Genre":[],"Book Review":[]}
    for book in soup.select('h4'):
        bookName.append(book.a.text)
    for author in soup.findAll('p',class_='sans bold'):
        authorName.append(author.text.replace('\n','').strip())
    for g in soup.findAll(class_="genre-links hidden-phone"):
        genre.append(g.text.replace('\n','').strip())
    for link in soup.findAll(class_='read-full'):
         reviewLink.append(link.a.get('href'))
    for i in range(0,5):
        reviewUrl=bookreviewurl.replace('reviews',reviewLink[i])
        response = requests.get(reviewUrl)
        #print(reviewUrl)
        reviewshtml = response.text
        soup=BeautifulSoup(reviewshtml)
        for rev in soup.findAll('div',class_="article-body"):
            reviews.append(str(rev.text).replace('\n',''))
        BookReviews["BookName"].append(bookName[i])
        BookReviews["Author"].append(authorName[i])
        BookReviews["Genre"].append(genre[i])
        BookReviews["Book Review"].append(reviews[i])
    df=pd.DataFrame(BookReviews)
    return df

d=bookReview('https://bookpage.com/reviews')
d.head()

,BookName,Author,Genre,Book Review
0,★ Festival Days,"Jo Ann Beard, Suehyla El-Attar",Audio / Nonfiction / Essays,"Essayist Jo Ann Beard explores life, death and..."
1,Listen,"Gabi Snyder, Stephanie Graegin",Children's / Children's Picture Book,"In the opening pages of Listen, a girl stands ..."
2,★ Rise to the Sun,Leah Johnson,YA / YA Fiction,"For Olivia Brooks, the three-day Farmland Musi..."
3,Filthy Animals,Brandon Taylor,Fiction / Short Stories,"In fiction, the corporal ecstasy of sexual ten..."
4,The Woman They Could Not Silence,Kate Moore,Nonfiction / History / Women's History,We’re all familiar with the method of discredi...


In [136]:
def scrape_Cric(cricHtml):
        response = requests.get(cricHtml)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        team=[]
        matches=[]
        points=[]
        rating=[]
        for t in soup.findAll('span',class_='u-hide-phablet'):
            team.append(t.text.replace('\n','').strip())
        for m in soup.findAll('td',class_='rankings-block__banner--matches'):
            matches.append(m.text.replace('\n','').strip())
        for p in soup.findAll('td',class_='rankings-block__banner--points'):
            points.append(p.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='rankings-block__banner--rating u-text-right'):
            rating.append(r.text.replace('\n','').strip())
        for m in soup.findAll('td',class_='table-body__cell u-center-text'):
            matches.append(m.text.replace('\n','').strip())
        for p in soup.findAll('td',class_='table-body__cell u-center-text'):
            points.append(p.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='table-body__cell u-text-right rating'):
            rating.append(r.text.replace('\n','').strip())
        
        OdiReviews=pd.DataFrame({
          "Team" :team[:10],
            "Matches":matches[:10],
            "points":points[:10],
            "Rating":rating[:10]
        }) 
        return OdiReviews
        
df=scrape_Cric('https://www.icc-cricket.com/rankings/mens/team-rankings/odi')
df

,Team,Matches,points,Rating
0,New Zealand,17,"2,054",121
1,Australia,25,25,118
2,India,"2,945","2,945",115
3,England,29,29,115
4,South Africa,"3,344","3,344",107
5,Pakistan,27,27,97
6,Bangladesh,"3,100","3,100",90
7,West Indies,20,20,82
8,Sri Lanka,"2,137","2,137",78
9,Afghanistan,24,24,62


In [137]:
df=scrape_Cric('https://www.icc-cricket.com/rankings/womens/team-rankings/odi')
df

,Team,Matches,points,Rating
0,Australia,18,"2,955",164
1,South Africa,24,24,118
2,England,"2,828","2,828",117
3,India,17,17,111
4,New Zealand,"1,993","1,993",93
5,West Indies,20,20,85
6,Pakistan,"2,226","2,226",73
7,Bangladesh,21,21,61
8,Sri Lanka,"1,947","1,947",47
9,Ireland,12,12,13


In [155]:
def scrape_CricPlayer(cricHtml):
        response = requests.get(cricHtml)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        playerName=[]
        team=[]
        rating=[]
        for name in soup.findAll('div',class_='rankings-block__banner--name-large'):
            playerName.append(name.text.replace('\n','').strip())
        for t in soup.findAll('div',class_='rankings-block__banner--nationality'):
            team.append(t.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='rankings-block__banner--rating'):
            rating.append(r.text.replace('\n','').strip())
        for name in soup.findAll('td',class_='table-body__cell rankings-table__name name'):
            playerName.append(name.text.replace('\n','').strip())
        for t in soup.findAll('span',class_='table-body__logo-text'):
            team.append(t.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='table-body__cell rating'):
            rating.append(r.text.replace('\n','').strip())
        #print(playerName)
        
        OdiReviews=pd.DataFrame({
          "playerName" :playerName[:10],
            "team":team[:10],
            "Rating":rating[:10]
        }) 
        return OdiReviews
        
bowlingMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling')
battingMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting')
battingMendf


,playerName,team,Rating
0,Babar Azam,PAK,857
1,Virat Kohli,IND,825
2,Rohit Sharma,IND,801
3,Ross Taylor,NZ,791
4,Aaron Finch,AUS,785
5,Jonny Bairstow,ENG,778
6,Fakhar Zaman,PAK,778
7,Francois du Plessis,SA,773
8,David Warner,AUS,773
9,Shai Hope,WI,756


In [152]:
bowlingMendf

,playerName,team,Rating
0,Trent Boult,NZ,713
1,Mehedi Hasan,BAN,708
2,Mujeeb Ur Rahman,AFG,691
3,Matt Henry,NZ,690
4,Jasprit Bumrah,IND,666
5,Kagiso Rabada,SA,665
6,Chris Woakes,ENG,660
7,Josh Hazlewood,AUS,646
8,Pat Cummins,AUS,645
9,Mustafizur Rahman,BAN,638


In [156]:
bowlingWoMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/womens/player-rankings/odi/bowling')
battingWoMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting')

In [157]:
bowlingWoMendf

,playerName,team,Rating
0,Jess Jonassen,AUS,762
1,Megan Schutt,AUS,747
2,Marizanne Kapp,SA,717
3,Shabnim Ismail,SA,681
4,Jhulan Goswami,IND,655
5,Katherine Brunt,ENG,641
6,Poonam Yadav,IND,638
7,Ayabonga Khaka,SA,616
8,Ellyse Perry,AUS,610
9,Shikha Pandey,IND,607


In [158]:
battingWoMendf

,playerName,team,Rating
0,Tammy Beaumont,ENG,758
1,Lizelle Lee,SA,756
2,Alyssa Healy,AUS,746
3,Stafanie Taylor,WI,723
4,Meg Lanning,AUS,715
5,Amy Satterthwaite,NZ,710
6,Smriti Mandhana,IND,709
7,Mithali Raj,IND,685
8,Natalie Sciver,ENG,683
9,Laura Wolvaardt,SA,679


In [201]:
def mob_data(amazonHtml):
    response = requests.get(amazonHtml)
    print(response.status_code)
    mobileHtml = response.text
    soup=BeautifulSoup(mobileHtml)
    mobileName=[]
    price=[]
    imageUrl=[]
    avgRating=[]
    for mob in soup.findAll('h2',class_='a-size-mini a-spacing-none a-color-base s-line-clamp-2'):
        mobileName.append(mob.text.replace('\n','').strip())
    for p in soup.findAll('span',class_='a-offscreen'):
        price.append(p.text.replace('\n','').strip())
    for i in soup.findAll('img',class_='s-image'):
        imageUrl.append(i['src'])
    for ar in soup.findAll('span',class_='a-icon-alt'):
        avgRating.append(ar.text)
    
    Mobiledata=pd.DataFrame({
          "mobileName" :mobileName[:10],
         "price":price[:10],
        "imageUrl":imageUrl[:10],
          "rating": avgRating[:10]
    }) 
    return Mobiledata

df=mob_data('https://www.amazon.in/s?k=mobiles+under+20000&rh=n%3A1389432031&dc&ref=a9_asc_1')
df

200


,mobileName,price,imageUrl,rating
0,"Samsung Galaxy M12 (Blue,4GB RAM, 64GB Storage...","₹10,999",https://m.media-amazon.com/images/I/71yYaNztZ0...,4.1 out of 5 stars
1,"Redmi 9A (Nature Green, 2GB RAM, 32GB Storage)...","₹12,999",https://m.media-amazon.com/images/I/71sxlhYhKW...,4.2 out of 5 stars
2,"Samsung Galaxy M11 (Metallic Blue, 4GB RAM, 64...","₹6,799",https://m.media-amazon.com/images/I/71GQUxuSpn...,4.2 out of 5 stars
3,"TECNO Spark 7T(Jewel Blue, 4GB RAM, 64GB Stora...","₹8,499",https://m.media-amazon.com/images/I/81aWyRY67S...,4.1 out of 5 stars
4,"Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 500...","₹9,499",https://m.media-amazon.com/images/I/71A9Vo1Bat...,4.2 out of 5 stars
5,"Redmi 9 Power (Mighty Black, 6GB RAM, 128GB St...","₹14,999",https://m.media-amazon.com/images/I/61LHaUOheh...,4.2 out of 5 stars
6,"Vivo Y51A (Titanium Sapphire, 8GB, 128GB Stora...","₹8,999",https://m.media-amazon.com/images/I/71BRHXBME2...,4.1 out of 5 stars
7,"Samsung Galaxy M31 (Ocean Blue, 8GB RAM, 128GB...","₹10,999",https://m.media-amazon.com/images/I/71-Su4Wr0H...,4.3 out of 5 stars
8,"Redmi 9 (Carbon Black, 4GB RAM, 64GB Storage) ...","₹8,799",https://m.media-amazon.com/images/I/716nHhG9SW...,4.2 out of 5 stars
9,"Redmi Note 10 (Aqua Green, 6GB RAM, 128GB Stor...","₹10,999",https://m.media-amazon.com/images/I/71-BcSc9rh...,4.1 out of 5 stars
